In [1]:
data_path = "./Data/starting_ki/train_all_tasks.csv"

In [2]:
import pandas as pd
data = pd.read_csv(data_path)
data = data[data["label_category"]!="none"]

In [3]:
label_category_map = {
    "1. threats, plans to harm and incitement":"Threat",
    "2. derogation":"Derogation",
    "3. animosity":"Animosity",
    "4. prejudiced discussions":"Prejudiced discussion",
    "Threat":"1. threats, plans to harm and incitement",
    "Derogation":"2. derogation",
    "Animosity":"3. animosity",
    "Prejudiced discussion":"4. prejudiced discussions"
}
print(label_category_map["4. prejudiced discussions"])

Prejudiced discussion


In [4]:
import random
# function returns two random text samples to use them as GPT-J prompt
def get_three_random_samples(data_frame):
    # get two random samples from `data file`
    s1, s2, s3 = random.sample(range(0, len(data)), 3)
    print(s1,s2,s3)
    # get text labels
    label1 = label_category_map[data_frame['label_category'].iloc[s1]]
    label2 = label_category_map[data_frame['label_category'].iloc[s2]]
    label3 = label_category_map[data_frame['label_category'].iloc[s3]]
#     print(label_category_map[label1])
    
    # return text1, label1, text2, label2
    return data_frame['text'].iloc[s1], label1, data_frame['text'].iloc[s2], label2, data_frame['text'].iloc[s3], label3


In [5]:
def get_prompt(text1, label1, text2, label2, text3, label3):
    # define a function that takes as input three samples and generates the prompt
    # that we should pass to the GPT-3 language model for completion.
    description = "Each item in the following list contains a sexist tweet and the respective category. Category is one of 'Threat', 'Derogation', 'Animosity', or 'Prejudiced discussion'"
    prompt = (f"{description}\n"
            f"Tweet: {text1} (Sentiment: {label1})\n"
            f"Tweet: {text2} (Sentiment: {label2})\n"
            f"Tweet: {text3} (Sentiment: {label3})\n"
            f"Tweet:")
    return prompt


In [6]:
a,b,c,d,e,f=get_three_random_samples(data)
get_prompt(a,b,c,d,e,f)

2022 503 1991


"Each item in the following list contains a sexist tweet and the respective category. Category is one of 'Threat', 'Derogation', 'Animosity', or 'Prejudiced discussion'\nTweet: I like 19 year old pussy. I know, call me crazy... (Sentiment: Derogation)\nTweet: #SoLikeUs I guess I failed as a parent when I didn't dress my daughter up in a tiara and have her and all her friends get humped for her 8th birthday. I guess this explains why 2/3rds of all negresses aged 11-17 have STD's. (Sentiment: Derogation)\nTweet: Chicks dig scars, maybe you should do a cool one your cheek. (Sentiment: Animosity)\nTweet:"

In [10]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer, AutoTokenizer, AutoModelForCausalLM, pipeline, set_seed
# tokenizer = GPT2Tokenizer.from_pretrained("gpt2-xl")
# model = GPT2LMHeadModel.from_pretrained("gpt2-xl", pad_token_id=tokenizer.eos_token_id)
tokenizer = AutoTokenizer.from_pretrained("ML-Projects-Kiel/tweetyface")

model = AutoModelForCausalLM.from_pretrained("ML-Projects-Kiel/tweetyface")


Downloading:   0%|          | 0.00/255 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/798k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/131 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/948 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/510M [00:00<?, ?B/s]

In [11]:
prompt_list=[]
res_list=[]

In [13]:
a,b,c,d,e,f=get_three_random_samples(data)
sentence = get_prompt(a,b,c,d,e,f)
prompt_list.append(sentence)
print(sentence)
print("===================================")
input_ids = tokenizer.encode(sentence, return_tensors='pt')
# generate text until the output length (which includes the context length) reaches 50
output = model.generate(input_ids, max_length=250, num_beams=5, no_repeat_ngram_size=2, early_stopping=True)
ans=tokenizer.decode(output[0], skip_special_tokens=True)
print(ans)
print("===================================")
                                                            
res_list.append(ans)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


1923 44 724
Each item in the following list contains a sexist tweet and the respective category. Category is one of 'Threat', 'Derogation', 'Animosity', or 'Prejudiced discussion'
Tweet: Alexandia Ocasio Cortez .. pretty gal should be in movies not in office (Sentiment: Derogation)
Tweet: [USER] most moms are no sex at all, from all the complaints I hear. (Sentiment: Animosity)
Tweet: And I thought 1960s hippies were ridiculous! They must be the offspring of slutty single mothers! They don't amass a lick of sense in their heads until they meet the real world which chews them up and spits them out! (Sentiment: Derogation)
Tweet:
Each item in the following list contains a sexist tweet and the respective category. Category is one of 'Threat', 'Derogation', 'Animosity', or 'Prejudiced discussion'
Tweet: Alexandia Ocasio Cortez.. pretty gal should be in movies not in office (Sentiment: Derogation)
Tweet: [USER] most moms are no sex at all, from all the complaints I hear. (Sentiment: Animosi